In [ ]:
import numpy as np
import os, sys
import cv2
sys.path.append('cython_hw/')
import lfit_cython
from tutils import *
from bokeh.plotting import figure,output_file,show
from bokeh.layouts import gridplot

In [ ]:
#target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190724 5min try3/'
target_folder='/Volumes/Users/Public/Sandhulab Data/Taisuke/20190809 control/'
all_files_for_analysis=extract_videos_for_processing(target_folder,extract_template=True)
#extract_temp_from_folder(target_folder+'pictures/')

In [ ]:
all_files_for_analysis=['/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/1m/08082019_141300_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/2m/08082019_141300_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/3m/08082019_141300_3m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/1m/08082019_142051_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/2m/08082019_142051_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/3m/08082019_142051_3m.mov']

In [ ]:
pp_list=[]
ffa=[]
for f in all_files_for_analysis:
    os.system("say 'Please select the dimensions of video'")
    fname=f #target_folder+f
    newname,pp_list=crop_and_trim(fname,pp_list)
    ffa.append(newname)
os.system("say 'Done'")

In [ ]:
#infile='/Users/Jaiyam/Dropbox/New Beads Videos/20190419 gravity/control.mov'
#outfile='/Users/Jaiyam/Dropbox/New Beads Videos/20190419 gravity/tracking/control/1m/control_1m.mov'
#crop_video(infile,outfile,46,56)
#extract_template_frames(all_files_for_analysis[1])

In [ ]:
for idx,fname in enumerate(ffa):
    template_file=target_folder+'temp1.jpg' #fname[:fname.rfind('/')+1]
    threshold=0.80
    try:
        track_video(fname,template_file,threshold)
        print('Video {}/{} finished tracking'.format(idx+1,len(all_files_for_analysis)))
    except Exception, e:
        print(str(e))
    print('\nDone')
os.system("say 'Done'")

# Post analysis of stopped beads with easy criterion

In [ ]:
ffa=['/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/1m/tracking/08082019_141300_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/2m/tracking/08082019_141300_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_141300/3m/tracking/08082019_141300_3m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/1m/tracking/08082019_142051_1m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/2m/tracking/08082019_142051_2m.mov',
 '/Volumes/Users/Public/Sandhulab Data/Taisuke/20190808 good/tracking/08082019_142051/3m/tracking/08082019_142051_3m.mov']

In [ ]:
target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190318 ALL/exp1/'
fname=target_folder+'tracking/exp1_100nM/1m/exp1_100nM_1m.mov'
txtfile=fname[:fname.rfind('.')]+'_data.txt'
total_frames=240
bead_radius=3
tracked_objs=[]
tracked_objs=load_beads(txtfile)
last_frame_of_video=get_last_frame(fname)
highlighted_easy,num_stopped_easy,stopped_idxs_easy=highlight_stopped_beads(last_frame_of_video,tracked_objs,total_frames,bead_radius,std_threshold=1.0,strict=False)
cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_stopped_beads_easy.jpg'.format(num_stopped_easy),highlighted_easy)
stoppedtxt_easy=txtfile[:txtfile.rfind('.')]+'_stopped_easy.txt'
save_beads(stoppedtxt_easy,[tracked_objs[idx] for idx in stopped_idxs_easy])
print('Total beads tracked= {}'.format(len(tracked_objs)))
if num_stopped_easy>0:
    text2csv(stoppedtxt_easy)

## Post analysis of beads stopped in sensing area

In [ ]:
all_files_for_analysis=['/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/1m/100nM_1m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/2m/100nM_2m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/3m/100nM_3m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/4m/100nM_4m.mov',
 '/Users/Jaiyam/Dropbox/New Beads Videos/20190702 long/tracking/100nM/5m/100nM_5m.mov']

In [ ]:
#target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190328 30mA 250G/'
#fname=target_folder+'tracking/control no NHS biotin on surface/1m/control no NHS biotin on surface_1m.mov'
oldres=None
failed=[]
succeeded=[]
for fname in all_files_for_analysis:
    txtfile=fname[:fname.rfind('.')]+'_data.txt'
    total_frames=240
    bead_radius=3
    tracked_objs=[]
    tracked_objs=load_beads(txtfile)
    debug=True
    try:
        highlighted_sa,num_stopped_sa,num_in_sa,oldres=find_beads_in_sensing_area(fname,tracked_objs,total_frames, bead_radius,strict=True,debug=debug,oldres=oldres)
        cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_stopped_beads_sa.jpg'.format(num_stopped_sa),highlighted_sa)
        stoppedtxt_sa=txtfile[:txtfile.rfind('.')]+'_stopped_sa.txt'
        #save_beads(stoppedtxt_sa,[tracked_objs[idx] for idx in stopped_idxs_sa])
        print('Total beads in sensing area= {}'.format(num_in_sa))
        with open(stoppedtxt_sa,'w') as f:
            msg='Number of beads in sensing area={}\n Number of beads stopped={}\n Percentage of beads stopped= {:.3f}\n'.\
            format(num_in_sa,num_stopped_sa,num_stopped_sa*100.0/num_in_sa)
            f.write(msg)
        succeeded.append(fname)
    except Exception, e:
        print('Error while analyzing file: {}'.format(fname))
        print(str(e))
        failed.append(fname)

    if len(succeeded)>0 and len(failed)>0:
        all_files_for_analysis.extend(failed) #modifying something while iterating over it. for shame!
        failed=[]

## Make graphs of bead paths

In [ ]:
target_folder='/Users/Jaiyam/Dropbox/New Beads Videos/20190710 tem/'
fname=target_folder+'tracking/20190705_121016-6ml/video.mov'
txtfile=fname[:fname.rfind('.')]+'_data.txt'
total_frames=35*15 #240
bead_radius=6 #3
tracked_objs=[]
tracked_objs=load_beads(txtfile)
good_beads=[62,4,25]#,14,47,2,3,59,25 #62,4
axis='y'
fps=15.0 #24.0
plot_pos_freq(tracked_objs,good_beads,fname[:fname.rfind('.')]+axis+'_pos.html',fs=fps,coord=axis)
last_frame=get_last_frame(fname)
cv2.imwrite(fname[:fname.rfind('.')]+'GBP.jpg',draw_full_paths_of_these_beads(last_frame,good_beads,tracked_objs))

In [ ]:
last_frame_of_video=get_last_frame(fname)
highlighted,num_stopped,stopped_idxs=highlight_stopped_beads(last_frame_of_video,tracked_objs,total_frames,bead_radius,std_threshold=2.0)
cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_stopped_beads.jpg'.format(num_stopped),highlighted)
save_beads(stoppedtxt,[tracked_objs[idx] for idx in stopped_idxs])
if num_stopped>0:
    text2csv(stoppedtxt)
with open(fname[:fname.rfind('/')+1]+'correspondence.txt','w') as f:
    f.write(str([b for b in enumerate(stopped_idxs)]))

In [ ]:
last_frame_of_video=get_last_frame(fname)
highlighted,num_stopped,stopped_idxs=highlight_stopped_beads(last_frame_of_video,tracked_objs,total_frames,bead_radius,std_threshold=2.0,end=144)
cv2.imwrite(fname[:fname.rfind('.')+1]+'_{}_initial_stopped_beads.jpg'.format(num_stopped),highlighted)
# save_beads(stoppedtxt,[tracked_objs[idx] for idx in stopped_idxs])
# if num_stopped>0:
#     text2csv(stoppedtxt)
# with open(fname[:fname.rfind('/')+1]+'correspondence.txt','w') as f:
#     f.write(str([b for b in enumerate(stopped_idxs)]))

In [ ]:
time_dict={idx: obj.tracked_frame_indices for idx,obj in enumerate(tracked_objs)}

In [ ]:
with open(fname[:fname.rfind('/')+1]+'frame_indices.txt','w') as f:
    f.write(str(time_dict))

In [ ]:
p=figure()
print(sum([b.is_oscillating for b in tracked_objs]))
good_beads=[] #[264,342,810,828,218,336,646]
colors=['red','green','blue','black','orange','firebrick','fuchsia','indigo','magenta']
num=0
for idx,b in enumerate(tracked_objs):
    if b.is_oscillating:
        b.show_fft(p,'y',color=colors[num],display=False)
        num+=1
        good_beads.append(idx)
    else:
        pass
        #print('{} is not oscillating'.format(b))
    if num==9:
        break
    
show(p)